In [1]:
import os   #work with folders & files
import re   #text pattern matching
import io   #handle image/text streams
import time
from datetime import datetime
from collections import Counter, defaultdict

import fitz  # PyMuPDF
from PIL import Image
import pytesseract   #Tesseract reads text from images

import tiktoken
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec   #stores embeddings (vector database)


In [2]:
pytesseract.pytesseract.tesseract_cmd =r'C:\tesseract\tesseract.exe'
print("✅ Tesseract path set:", pytesseract.pytesseract.tesseract_cmd)

✅ Tesseract path set: C:\tesseract\tesseract.exe


In [ ]:
openai_api_key = "Paste Your OpenAI API Key"
PINECONE_API_KEY = "Paste Your Pinecone API Key" 
client = OpenAI(api_key=openai_api_key)
pc = Pinecone(api_key=PINECONE_API_KEY)

print("✅ Clients ready")

✅ Clients ready


In [ ]:
index_name = "fda483-rag-v1-clean" # Pinecone Index Name

# delete if exists (optional)
"""existing = [i["name"] for i in pc.list_indexes()]
if index_name in existing:
    pc.delete_index(index_name)
    print("🗑️ Deleted old index")

pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)"""

index = pc.Index(index_name)
print("✅ Pinecone index ready:", index_name)


✅ Pinecone index ready: fda483-rag-v1-clean


In [5]:
#Text Converts into numbers (vector) - Embedding

def get_embedding(text: str):
    text = text.replace("\n", " ")
    res = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return res.data[0].embedding


In [6]:
#Removes too many line breaks and extra spaces

def clean_text(text: str):
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


In [7]:
#PDF → image → OCR → text

def ocr_page(pdf_path, page_number, dpi=300):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)
    pix = page.get_pixmap(dpi=dpi)
    img_bytes = pix.tobytes("png")
    doc.close()

    img = Image.open(io.BytesIO(img_bytes))
    text = pytesseract.image_to_string(img)

    return clean_text(text)


In [8]:
# Extract full page text and if blank than use OCR


def extract_full_text(pdf_path):
    doc = fitz.open(pdf_path)
    parts = []

    for p in range(len(doc)):
        page = doc.load_page(p)
        text = clean_text(page.get_text("text"))

        # if scanned or empty text -> OCR
        if len(text) < 40:
            print(f"⚠️ OCR: {os.path.basename(pdf_path)} page {p+1}")
            text = ocr_page(pdf_path, p)

        if len(text) >= 40:
            parts.append(f"\n\n--- PAGE {p+1} ---\n\n{text}")

    doc.close()
    return "\n".join(parts)


In [9]:
# uses to extract company name and remove dates and numbers from the pdf name

def extract_firm_name_from_filename(filename: str):
    name = filename.lower().replace(".pdf", "")
    name = name.replace("_", " ").replace("-", " ")

    # remove date patterns + numbers
    name = re.sub(r"\d{1,2}\.\d{1,2}\-\d{1,2}\.\d{1,2}\.\d{2}", " ", name)
    name = re.sub(r"\d+", " ", name)

    # remove common words
    for w in ["redacted", "final", "sb", "483"]:
        name = name.replace(w, " ")

    name = re.sub(r"\s+", " ", name).strip()

    # take first 6 words
    return " ".join(name.split()[:6])


In [10]:
#Splits Observations

def split_fda_sections(full_text: str):
    full_text = clean_text(full_text)
    pattern = r"\bOBSERVATION\s+\d+\b"
    matches = list(re.finditer(pattern, full_text, flags=re.IGNORECASE))

    if not matches:
        return [("DOCUMENT", full_text)]

    sections = []
    header = full_text[:matches[0].start()].strip()
    if header:
        sections.append(("DOCUMENT", header))

    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i+1].start() if i+1 < len(matches) else len(full_text)
        title = matches[i].group().upper()
        body = full_text[start:end].strip()
        sections.append((title, body))

    return sections

def detect_observation_number(section_title: str):
    m = re.search(r"OBSERVATION\s+(\d+)", section_title, re.IGNORECASE)
    return int(m.group(1)) if m else None


In [11]:
# Chunks Creation

def make_chunks(text, chunk_size=1200, overlap=200):
    chunks = []
    start = 0
    step = chunk_size - overlap

    while start < len(text):
        end = start + chunk_size
        ch = text[start:end].strip()
        if ch:
            chunks.append(ch)
        start += step

    return chunks


In [12]:
# Final chunk creation sections combines observation sections, firm name, source file

def build_fda_chunks(full_text, source):
    sections = split_fda_sections(full_text)
    firm_name = extract_firm_name_from_filename(source)

    final_chunks = []
    chunk_id = 0

    for title, section_text in sections:
        obs_num = detect_observation_number(title)
        section_chunks = make_chunks(section_text, chunk_size=1200, overlap=200)

        for part_no, ch in enumerate(section_chunks, start=1):
            chunk_id += 1

            metadata = {
                "text": ch,
                "source": source,
                "firm_name": firm_name,
                "section": title,
                "section_type": "observation" if obs_num is not None else "header"
            }

            if obs_num is not None:
                metadata["observation_number"] = str(obs_num)

            final_chunks.append({
                "id": f"{source}_{title}_part{part_no}_{chunk_id}",
                "values": get_embedding(ch),
                "metadata": metadata
            })

    return final_chunks


In [13]:
# Upload chunks to pinecone

def upsert_chunks_to_pinecone(vectors, batch_size=50):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        index.upsert(vectors=batch)
        time.sleep(0.15)


In [14]:
#Create a time loop while reading the file and extracting chunks

def log_progress(i, total, filename, chunk_count, start_time):
    elapsed = time.time() - start_time
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now}] ✅ PDF {i}/{total} | file={filename} | chunks={chunk_count} | time={elapsed:.1f}s")


In [ ]:
# Load pdf folder

pdf_folder = r"C:\Users\user\FDA_RAG_BOT\fda_483_docs" # Path of your folder where you kept all your pdf
pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]
total = len(pdf_files)

print("✅ PDFs found:", total)

for i, f in enumerate(pdf_files, start=1):
    start_time = time.time()
    pdf_path = os.path.join(pdf_folder, f)

    print(f"\n📄 Processing {f}")
    full_text = extract_full_text(pdf_path)
    vectors = build_fda_chunks(full_text, source=f)

    upsert_chunks_to_pinecone(vectors)

    log_progress(i, total, f, len(vectors), start_time)

print("✅ Done uploading all PDFs")
print(index.describe_index_stats())


✅ PDFs found: 56

📄 Processing alvotech_hf_rekjavik_iceland_6.26-7.4.25_483_redacted.pdf
[11:06:22] ✅ PDF 1/56 | file=alvotech_hf_rekjavik_iceland_6.26-7.4.25_483_redacted.pdf | chunks=51 | time=75.8s

📄 Processing applied_483_682998_redacted.508.pdf
[11:06:30] ✅ PDF 2/56 | file=applied_483_682998_redacted.508.pdf | chunks=12 | time=7.5s

📄 Processing applied_483_691575_redacted.508.pdf
[11:07:21] ✅ PDF 3/56 | file=applied_483_691575_redacted.508.pdf | chunks=41 | time=50.6s

📄 Processing applied_anazaohealth_corporation_3011152407_483_08082025_sb_redacted.508.pdf
[11:07:29] ✅ PDF 4/56 | file=applied_anazaohealth_corporation_3011152407_483_08082025_sb_redacted.508.pdf | chunks=15 | time=8.2s

📄 Processing applied_bso_llc_3011976853_483_07162025_kum_redacted.508.pdf
[11:07:47] ✅ PDF 5/56 | file=applied_bso_llc_3011976853_483_07162025_kum_redacted.508.pdf | chunks=36 | time=18.6s

📄 Processing applied_cw_osrx_inc_3014549846_483_111425_redacted.508.pdf
[11:07:58] ✅ PDF 6/56 | file=applied

In [16]:
#Firm detection from simple name

def load_firm_pdf_map(sample_k=5000):
    dummy = get_embedding("firm index build")
    res = index.query(vector=dummy, top_k=sample_k, include_metadata=True)

    firm_map = defaultdict(set)
    for m in res["matches"]:
        md = m.get("metadata", {})
        firm = md.get("firm_name")
        src = md.get("source")
        if firm and src:
            firm_map[firm].add(src)

    return {k: sorted(list(v)) for k, v in firm_map.items()}

FIRM_PDF_MAP = load_firm_pdf_map()
print("✅ Firms loaded:", len(FIRM_PDF_MAP))


✅ Firms loaded: 55


In [17]:
#Firm detection from simple name

def detect_firm_and_pdf(question: str):
    q = question.lower()
    q_tokens = set(re.findall(r"[a-z0-9]+", q))

    best_firm = None
    best_score = 0

    for firm, pdfs in FIRM_PDF_MAP.items():
        firm_tokens = set(firm.split())
        score = len(q_tokens.intersection(firm_tokens))

        if score > best_score:
            best_score = score
            best_firm = firm

    # require at least 2 matching tokens for high precision
    if best_firm and best_score >= 2:
        # choose latest pdf (or first)
        return best_firm, FIRM_PDF_MAP[best_firm][0]

    return None, None


In [18]:
#Retrieval

def retrieve_context(question, pdf_name=None, top_k=12):
    query_emb = get_embedding(question)

    if pdf_name:
        res = index.query(
            vector=query_emb,
            top_k=top_k,
            include_metadata=True,
            filter={"source": {"$eq": pdf_name}}
        )
    else:
        res = index.query(
            vector=query_emb,
            top_k=top_k,
            include_metadata=True
        )

    contexts = []
    for m in res["matches"]:
        md = m.get("metadata", {})
        txt = md.get("text", "")
        src = md.get("source", "unknown")
        sec = md.get("section", "unknown")
        if txt.strip():
            contexts.append(f"(Source: {src}, Section: {sec})\n{txt}")

    return "\n\n---\n\n".join(contexts)


In [19]:
# Specially for observations retrieval

def retrieve_all_observations(pdf_name, top_k=250):
    query_emb = get_embedding("OBSERVATION 1 OBSERVATION 2 OBSERVATION 3")

    res = index.query(
        vector=query_emb,
        top_k=top_k,
        include_metadata=True,
        filter={
            "source": {"$eq": pdf_name},
            "section_type": {"$eq": "observation"}
        }
    )

    contexts = []
    for m in res["matches"]:
        md = m.get("metadata", {})
        txt = md.get("text", "")
        sec = md.get("section", "")
        if txt.strip():
            contexts.append(f"(Source: {pdf_name}, {sec})\n{txt}")

    return "\n\n---\n\n".join(contexts)


In [20]:
# Context Trimmer

def trim_context(context: str, max_chars=160000):
    return context[:max_chars] if len(context) > max_chars else context


In [21]:
# Cost Calculation

PRICES = {"gpt-4o-mini": {"input": 0.15, "output": 0.60}}

def calc_cost(model, input_tokens, output_tokens):
    p = PRICES.get(model)
    return (input_tokens/1_000_000)*p["input"] + (output_tokens/1_000_000)*p["output"]

def chat_with_usage(system_prompt, user_prompt, model="gpt-4o-mini"):
    res = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0
    )

    usage = res.usage
    in_tok = usage.prompt_tokens
    out_tok = usage.completion_tokens
    total_tok = usage.total_tokens
    cost = calc_cost(model, in_tok, out_tok)

    return res.choices[0].message.content, in_tok, out_tok, total_tok, cost


In [22]:
SYSTEM_PROMPT = """
You are a strict compliance QA assistant for FDA 483 documents.

CRITICAL RULES (must follow):
1) Use ONLY the provided CONTEXT from the PDFs. Do NOT use external knowledge, assumptions, or guesses.
2) If the answer is not explicitly stated in the context, respond exactly: "Not found in the provided PDFs."
3) Do NOT paraphrase numbers, dates, names, or regulatory text. Copy them exactly as written.
4) Be COMPLETE: if the question asks for observations/details, extract ALL relevant items from the context (do not miss any).
5) Every claim must be supported by the context. If any claim is uncertain, mark it as "Not found in the provided PDFs."
6) Before finalizing, double-check that your answer is fully supported by the context and that no key point is missing.
7) Always include citations in the form: (Source: <file>, Page: <page>).
8) Check all the pages for every set of question, the answer should have all the deatils regarding to the question asked to the bot.
9) Don't skip any part,i.e. if there are 10 observation in a pdf than you must include all the observations. 
I need all the observation/deatils if they are present in the pages of that particular pdf
10) if question contains penalty/CAPA/NAI/VAI/OAI classification keywords and no match → don’t just say not found; give expert explanation.

OUTPUT RULE:
- Be precise and factual.
- Prefer structured answers with bullet points and headings.
- If the question is about observations, list them as Observation 1, Observation 2, etc. exactly as in the PDF text.

CONTEXT:
{context}

QUESTION:
{query}

ANSWER:
"""

In [39]:
#Detect Observation Questions

def is_observation_question(q: str):
    q = q.lower()
    return any(x in q for x in [
        "all observations", "list observations", "provide observations",
        "what are the observations", "extract observations","return all the observations","all the subpoints A to Z"
    ])

#RAG_Bot
def rag_bot(question, top_k=12):
    firm, pdf_name = detect_firm_and_pdf(question)

    obs_mode = is_observation_question(question)

    if obs_mode and pdf_name:
        context = retrieve_all_observations(pdf_name, top_k=250)
    else:
        context = retrieve_context(question, pdf_name=pdf_name, top_k=top_k)

    if not context.strip():
        return "Not found in the provided PDFs."

    context = trim_context(context)

    prompt = f"""
CONTEXT:
{context}

QUESTION:
{question}

Answer with citations (Source + Section). No hallucination.
"""

    answer, in_tok, out_tok, total_tok, cost = chat_with_usage(
        SYSTEM_PROMPT, prompt, model="gpt-4o-mini"
    )

    detected = f"✅ Detected firm: {firm} | PDF: {pdf_name}" if pdf_name else "✅ No firm detected (global search)"

    return f"""{detected}

✅ Answer:
{answer}

--------------------------
📊 Usage:
Input tokens: {in_tok}
Output tokens: {out_tok}
Total tokens: {total_tok}
Estimated cost (USD): {cost:.6f}
"""


In [41]:
#print(rag_bot("List all observations with all the subpoints in cohance lifesciences. Return all observation fully with all the sub groups in Cohance lifescience. Additionally provide me what steps as a new"))
#print(rag_bot("List all observations with all the subpoints(A,B,C,D,E,F) in cohance lifesciences. Return all observation fully with all the sub groups in Cohance lifescience. additionally, Study these observations and tell me what things we should avoide to have a successful fda quality check. "))#print(rag_bot("Is there any reference to CAPA effectiveness checks? Quote the lines. Scan for the pdf for this word 'CAPA' and provide me all the deatils with CAPA word in all the pdfs which you have in the system from provide the source."))
print(rag_bot("Return Some 10 common observations fully from all the pdfs which you have with all the subpoints without summarizing"))


✅ No firm detected (global search)

✅ Answer:
ANSWER:

### Observation 1
- **Observation**: The number of containers to be sampled is not based upon appropriate criteria.
  - Specifically, your firm samples [= container per lot of incoming raw materials and excipients, regardless of the total number of containers received. You have not established documented rationale or scientific justification to support this practice. The current sampling procedure does not provide assurance that samples are representative of the entire lot when multiple containers are received.  
  (Source: sumitomo_pharma_co._ltd._suzuka_japan_8.3-8.7.25_483_redacted.pdf, Section: OBSERVATION 1)

### Observation 2
- **Observation**: Written documents are inadequate to ensure proper documentation and analysis of critical manufacturing operations.
  - Specifically, Report VALRP-BTO-002395, Summary report for airflow visualization study and assessment of aseptic interventions during vial filling (Version 1.0, approve